In [ ]:

!pip install -q ultralytics

from ultralytics import YOLO
from google.colab import drive
import zipfile, os, shutil

#Mount Google Drive
drive.mount('/content/drive')

#Paths
zip_path = "/content/drive/MyDrive/emergency.zip"
extract_path = "/content/dataset_emergency"
save_dir = "/content/training_results/Emergency_Vehicles_Model_Swapped"
drive_output_dir = "/content/drive/MyDrive/Trained_Models"

#Extract dataset
print("Extracting dataset... please wait...")
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Dataset extracted successfully!")

#Verify structure
print("\Extracted folders:")
print(os.listdir(extract_path))

#Find data.yaml
data_yaml_path = None
for root, dirs, files in os.walk(extract_path):
    if "data.yaml" in files:
        data_yaml_path = os.path.join(root, "data.yaml")
        break

if not data_yaml_path:
    raise FileNotFoundError(" data.yaml not found inside the dataset zip!")

print(f"\n📄 Found data.yaml at: {data_yaml_path}")

#Start training
print("\n🚀 Starting YOLOv8 training...")
model = YOLO("yolov8m.pt")
model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=8,
    name="Emergency_Vehicles_Model_Swapped",
    project="/content/training_results",
    device=0  # GPU if available
)

#Zip and save trained model
zip_path = "/content/Emergency_Vehicles_Model_Swapped.zip"
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', save_dir)

os.makedirs(drive_output_dir, exist_ok=True)
shutil.copy(zip_path, os.path.join(drive_output_dir, "Emergency_Vehicles_Model_Swapped.zip"))

print(f"\n Model saved to Drive: {drive_output_dir}/Emergency_Vehicles_Model_Swapped.zip")
print(" Training complete and safely stored in Drive!")
